<div id="colab_button">
  <h1>Quick tour</h1>
  <a target="_blank" href="https://colab.research.google.com/github/mithril-security/blindbox/blob/main/docs/docs/getting-started/quick-tour.ipynb"> 
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
</div>
______________________________

## Introduction
______________________________


**BlindBox** is a **privacy solution** allowing developers to deploy their **Software-as-a-Service** (SaaS) applications, while **keeping** their users' data **confidential**. We use **secure enclaves**, hardware based isolated environments, to **protect data during computation**. What happens in the enclave, stays in the enclave - hence our name, BlindBox!

### Key Features 

⌛ *COMING SOON*

* Easy-to-use **CLI tool** for **deploying app images** in secure enclaves
* **Data** processed within a **hardware-based, highly isolated environments** called secure enclaves
* Additional customizable layer of isolation provided

## Quick start guide
______________________

While we continue to develop BlindBox's key features, we have created this guide to demonstrate how you can deploy and query the **OpenChatKit** (ChatGPT-like) model  with privacy guarantees using BlindBox.

### Installation

First of all, we need to install the [blindbox](https://pypi.org/project/blindbox/) Python library.

In [ ]:
# install blindbox
!pip install blindbox[ai]

### Preparing an image

BlindBox is a wrapper to make Docker images deployable inside secure enclaves and ensure isolation of the content.

**Requirements:**

BlindBox currently assumes your Docker image exposes port XXXX. 

We have packaged a Docker image that contains a Web server [to deploy a ChatGPT-like model](LINK TO OUR IMAGE).


### Build enclave image

Next, we will create our blindbox with our desired configuration TEE option supplied as an `--env` argument. Currently, We can chose between `azure-sev`, an AMD SEV-SNP TEE environment running within an Azure VM, or `aws-nitro`, a Nitro Enclave TEE running on an AWS instance. 

Once we have done this, we are ready to build our ChatGPT-like model Docker image with BlindBox.

In [ ]:
# create a blindbox configuration using the AMD SEV TEE with an Azure VM
!blindbox init --env azure-sev

# build the image
!blindbox build -t myimage-blindbox:v1 --source-image myimage:v1

### Deploy enclave image

To deploy our image within our secure enclave, we use the deploy command.

In [ ]:
# deploy our image inside secure enclave using BlindBox

!blindbox deploy

### Querying enclave

We can now query the model within the enclave using the `blindbox.requests` module.

In [ ]:
from blindbox.requests  import SecuredSession

# We create a secure session connecting to our enclave
with SecuredSession("http://localhost:8080") as secure_session:
    
    # we convert our input text to a JSON object using json.dumps
    req = json.dumps({
            "input_text": "He hoped there would be stew for dinner, turnips and carrots and bruised potatoes and fat-mutton pieces to be ladled out and thick, peppered flower-fattened sauce.",
    })

    # we query the OpenChatKit predict feature using our input text
    res = secure_session.post(endpoint="/open-chat-kit/predict", data=req).text # .text specifies we want the return result as a string
    
    # display result
    print(res)

## How to verify BlindBox's security features
___________________________________________

- Our source code is `open source`. You can inspect the code yourself on our [GitHub page](https://github.com/mithril-security/blindbox).

- We aim to provide clear explanations of the technologies behind BlindBox. You can get started with [an introduction to confidential computing](https://blindbox.mithrilsecurity.io/en/latest/docs/getting-started/confidential_computing/) explaining key concepts behind BlindBox, or go with more specific explanations in our [secure enclaves guide](https://blindbox.mithrilsecurity.io/en/latest/docs/concepts/Trusted_Execution_Environments/).

Also good to know: our historical product, **BlindAI**, has been **independently audited** by [Quarkslab SAS](https://www.quarkslab.com/fr/accueil/). The audit report is *coming soon*!

## Community
_________________

Ask questions on our [**Github**](https://github.com/mithril-security/blindbox/issues) or on our [**Discord**](https://discord.com/invite/TxEHagpWd4). We love feedback!🎉